In [2]:
import awswrangler as wr
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
BUCKET_NAME = os.getenv("bucket_name")
redshift_db = os.getenv("database")
redshift_user = os.getenv("user")
redshift_password = os.getenv("password")
redshift_host = os.getenv("host")

## COPY DATA FROM S3 TO REDSHIFT

In [52]:
s3_path=f"s3://{BUCKET_NAME}/green_tripdata_2021-01.parquet"

In [55]:
s3_data = wr.s3.read_parquet(path=s3_path)

# drop columns with NaN. redshift does not allow data witn NaN
s3_data = s3_data.drop(["payment_type","trip_type","congestion_surcharge","ehail_fee","store_and_fwd_flag",
"RatecodeID","PULocationID","DOLocationID","passenger_count"], axis=1)
s3_data.isna().sum()


VendorID                 0
lpep_pickup_datetime     0
lpep_dropoff_datetime    0
trip_distance            0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

In [57]:

conn = wr.redshift.connect("AWS_GLUE_REDSHIFT_CONNECTION")
wr.redshift.to_sql(
    df=s3_data,
    table="trip_data",
    schema="public",
    con=conn
)
conn.close()

In [58]:
s3_data.shape

(76518, 11)

## MOVE DATA FROM REDSHIFT TO S3

In [61]:
conn = wr.redshift.connect("AWS_GLUE_REDSHIFT_CONNECTION")
wr.redshift.unload(
    sql="SELECT * FROM public.trip_data",
    con=conn,
    #iam_role=iam_role,
    path=f"s3://{BUCKET_NAME}/redshift.parquet",
    keep_files=True,
)
conn.close()

## LIST ALL FILES IN S3 BUCKET

In [ ]:
for file in wr.s3.list_objects(path=f"s3://{BUCKET_NAME}"):
    print(file.split("/")[-1])

In [7]:
csv_data=wr.s3.read_csv(path=f"s3://{BUCKET_NAME}/SOE_4th_22nd_Nov_2022.csv")
csv_data.shape

(1589, 2)

In [8]:
csv_data.columns

Index(['time', 'event'], dtype='object')

In [10]:
csv_data.head()

,time,event
0,45:38.4,OFEX 132 GTX14 CB Open
1,46:03.4,OFEX 132 GTX14 CB Closed
2,47:16.7,OFEX 132 GTX14 CB Open
3,49:46.9,OFEX 132 GTX14 CB Closed
4,51:08.2,OFEX 132 GTX14 CB Open


In [11]:
wr.s3.to_parquet(path=f"s3://{BUCKET_NAME}/SOE_4th_22nd_Nov_2022.parquet", df=csv_data)

{'paths': ['s3://airflow-jkimera-bucket/SOE_4th_22nd_Nov_2022.parquet'],
 'partitions_values': {}}